## Spectrum approximation experiment (Section 5.2)

The script compares how close is the spectrum of a coarse graph to that of the original graph. 

The code accompanies the preprint: [Graph reduction with spectral and cut guarantees](https://arxiv.org/abs/1808.10650). The final version of the paper will appear at JMLR.

Depedencies: pygsp, matplotlib, numpy, scipy, networkx, sortedcontainers

This work was kindly supported by the Swiss National Science Foundation (grant number PZ00P2 179981).

15 March 2019

[Andreas Loukas](https://andreasloukas.wordpress.com)

Released under the MIT license 


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
from libraries.coarsening_utils import *
import libraries.graph_lib as graph_lib
import libraries.graph_utils as graph_utils

import numpy as np
import scipy as sp
from scipy import io
from scipy.linalg import circulant
import time
import os 

import matplotlib
import matplotlib.pylab as plt

import pygsp as gsp
gsp.plotting.BACKEND = 'matplotlib'

### Parameters

In [ ]:
graphs  = ['yeast', 'airfoil', 'minnesota', 'bunny'] 
methods = ['heavy_edge', 'variation_edges', 'variation_neighborhoods', 'algebraic_JC', 'affinity_GS', 'kron'] 
K_all   = np.array([10,40], dtype=np.int32)  
r_all   = [0.3, 0.5, 0.7]

print('k: ', K_all, '\nr: ', r_all)

### The actual experiment code (this will take long)
If one needs to just see the results, skip running this part.

In [ ]:
rerun_all = False
rewrite_results = False
if rerun_all:
    
    algorithm  = 'greedy'  
    max_levels = 10
    n_methods  = len(methods)
    n_graphs   = len(graphs)

    flag = (K_all[-1] == -1)

    for graphIdx, graph in enumerate(graphs):

        N = 4000

        if graph == 'bunny':
            G = graph_lib.real(N, 'bunny')
        elif graph == 'swissroll':
            G = graph_lib.knn(N, 'swissroll')
        elif graph == 'barabasi-albert':
            G = graph_lib.models(N, 'barabasi-albert')
        elif graph == 'block':
            G = graph_lib.clusterable(N, 'block', K=10, p = 10/N, q = 0.5/N) # works
        elif graph == 'regular':
            G = graph_lib.models(N, 'regular', k=10) 
        elif graph == 'grid':
            N1 = int(np.sqrt(N))
            G = graphs.Grid2d(N1=N1, N2=N1) # large r: edge-based better for moderate K, then heavy edge, small r: edge/neighborhood-based
        else:
            G = graph_lib.real(N, graph)     

        N = G.N

        if flag: 
            kmax = int(np.floor(N*(1-max(r_all))))-1
        else:
            kmax = max(K_all)

        # precompute spectrum needed for metrics
        if kmax > N/2:
            [Uk,lk] = eig(G.L)             
        else:
            offset = 2*max(G.dw)
            T = offset*sp.sparse.eye(G.N, format='csc') - G.L
            lk, Uk = sp.sparse.linalg.eigsh(T, k=kmax, which='LM', tol=1e-6)
            lk = (offset-lk)[::-1]
            Uk = Uk[:,::-1]                

        G.estimate_lmax()
        lambda_max = G.lmax

        eigenvalue = np.zeros((n_methods, len(K_all), len(r_all))) 
        ratio      = np.zeros((n_methods, len(K_all), len(r_all)))

        for rIdx,r in enumerate(r_all):     

            n_target = int(np.floor(N*(1-r)))
            if flag: K_all[-1] = int(np.floor(N*(1-r)))-1

            for KIdx, K in enumerate(K_all):

                print('{} {}| K:{:2.0f}'.format(graph, N, K))

                if K > n_target:
                    print('Warning: K={}>n_target={}. skipping'.format(K, n_target))
                    continue  

                for methodIdx,method in enumerate(methods):

                    # algorithm is not deterministic: run a few times
                    if method == 'kron':
                        if KIdx == 0:
                            n_iterations = 2
                            n_failed = 0
                            r_min = 1.0
                            for iteration in range(n_iterations):

                                Gc, iG  = kron_coarsening(G, r=r, m=None)
                                metrics = kron_quality(iG, Gc, kmax=K_all[-1], Uk=Uk[:,:K_all[-1]], lk=lk[:K_all[-1]])

                                if metrics['failed']: n_failed += 1
                                else:
                                    r_min = min(r_min, metrics['r'])
                                    for iKIdx, iK in enumerate(K_all):
                                        eigenvalue[methodIdx, iKIdx, rIdx] += np.nanmean(metrics['error_eigenvalue'][:iK]) 

                            eigenvalue[methodIdx, :, rIdx] /= (n_iterations-n_failed)
                            ratio[     methodIdx, :, rIdx]  = r_min

                            if np.abs(r_min - r) > 0.02: print('Warning: ratio={} instead of {} for {}'.format(r_min, r, method))

                    else:
                        C, Gc, Call, Gall = coarsen(G, K=K, r=r, max_levels=max_levels, method=method, algorithm=algorithm, Uk=Uk[:,:K], lk=lk[:K])
                        metrics = coarsening_quality(G, C, kmax=K, Uk=Uk[:,:K], lk=lk[:K])

                        eigenvalue[methodIdx, KIdx, rIdx] = np.nanmean(metrics['error_eigenvalue']) 
                        ratio[methodIdx, KIdx, rIdx]  = metrics['r']

                        if np.abs(metrics['r'] - r) > 0.02: 
                            print('Warning: ratio={} instead of {} for {}'.format(metrics['r'], r, method))


        if rewrite_results:
            filepath = os.path.join('.', 'results', 'experiment_spectrum_'+ graph +'.npz')
            print('.. saving to "' + filepath + '"')    
            np.savez(filepath, methods=methods, K_all=K_all, r_all=r_all, eigenvalue=eigenvalue, ratio=ratio)

print('done!') 

### General code for nice printing

In [ ]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

graphs  = ['yeast', 'airfoil', 'minnesota', 'bunny']

### Show all results as an ASCI table

In [ ]:
latex = False

sep = '&' if latex else ','

for KIdx,K in enumerate(K_all):     
    print('\n%--------------------------------------------------------------------')
    print(f'% K: {K}:')    
    print('%--------------------------------------------------------------------')

    if latex:
        string = 'r'
        for i in range(16): string += 'C{4mm}'
        print('\\begin{table}[]\n\\scriptsize\\centering\n\\begin{tabular}{' + string + '}\n\\toprule')

    # graph title line
    line = ''
    for graphIdx, graph in enumerate(graphs):        
        if latex :
            line = '{}\\multicolumn{{3}}{{c}}{{{:}}}{}'.format(line, graph,sep)
        else:
            line = '{} {:21s}   ,  '.format(line, graph)
    line = line[:-1]
    print('{0:18} {1} {2} \\\\'.format(' ', sep, line)) # \multicolumn{3}{c}{minesotta}

    if latex: print('\\cmidrule(l){2-16} ')
    
    # reduction title line
    line = '{0:18} {1} '.format(' ', sep)
    for graphIdx, graph in enumerate(graphs):        
        for rIdx, r in enumerate(r_all):
            line = '{}{:4.0f}\\% {} '.format(line, 100*r,sep)
        line = '{}{:1s}'.format(line, ' ')
    line = line[:-3]
    print('{}\\\\'.format(line))
    
    for methodIdx,method in enumerate(methods):
        
        method = method.replace('_', ' ')                
        if method == 'heavy edge':
            method = 'heavy edge'
        elif 'variation edges' in method:
            method = 'local var. (edges)'
        elif (method == 'variation neighborhoods') or (method == 'variation neighborhood'):
            method = 'local var. (neigh)'
        elif 'algebraic' in method:
            method = 'algebraic dist.'
        elif 'affinity' in method:
            method = 'affinity'
        elif method == 'kron':
            method = 'kron'
        else:
            continue

        # will hold one string per graph
        strings = []
        
        # for each graph
        for graphIdx, graph in enumerate(graphs):        
            filepath = os.path.join('.', 'results', 'experiment_spectrum_'+ graph +'.npz')
            data = np.load(filepath)
            eigenvalue = data['eigenvalue']
            # eigenvalue *= lmax[graphIdx]
            
            # for each r
            string = ''
            for rIdx, r in enumerate(r_all):
                if min(eigenvalue[:,KIdx,rIdx]) == eigenvalue[methodIdx,KIdx,rIdx]:
                    if latex:
                        string = '{} \\textbf{{{:0.3f}}} &'.format(string,  eigenvalue[methodIdx,KIdx,rIdx])
                    else:
                        string = '{} {}{:0.4f}{} ,'.format(string, color.BOLD, eigenvalue[methodIdx,KIdx,rIdx], color.END)
                else:
                    if latex:
                        string = '{} {:0.3f} {}'.format(string, eigenvalue[methodIdx,KIdx,rIdx], sep)
                    else:
                        string = '{} {:0.4f} {}'.format(string, eigenvalue[methodIdx,KIdx,rIdx], sep)
                        
            strings.append(string)
        
        combined = ' '.join(s for s in strings) 
        
        print('{0:18s} {2}{1} \\\\'.format(method, combined[:-2], sep))
        
    if latex: print('\\bottomrule\n\\end{tabular}\n\\end{table}')

### Measure error improvement

In [ ]:
measure = np.zeros((len(graphs), len(K_all), 2))*np.NaN
print('===========================================================')
for KIdx, K in enumerate(K_all):

    for graphIdx, graph in enumerate(graphs): 
    
        filepath = os.path.join('.', 'results', 'experiment_spectrum_'+ graph +'.npz')
        data = np.load(filepath)
        eigenvalue = data['eigenvalue']

        measure[graphIdx,KIdx,0] = np.min(eigenvalue[[0,3,4,5],KIdx,-1]) / np.min(eigenvalue[:,KIdx,-1],0)
        measure[graphIdx,KIdx,1] = np.min(eigenvalue[[0,3,4],  KIdx,-1]) / np.min(eigenvalue[:,KIdx,-1],0)
        print('  {:10} K:{}, with Kron:{:1.3f}, without Kron:{:1.3f}'.format(graph, K, measure[graphIdx,KIdx,0], measure[graphIdx,KIdx,1]))

    print('For this k: ' + str(np.nanmean(measure[:,KIdx,0])) + '/' + str(np.nanmean(measure[:,KIdx,1])))
    print('-----------------------------------------------------------')

print('===========================================================')
print('Overall:')
print(str(np.nanmean(measure[:,:,0])) + '/' + str(np.nanmean(measure[:,:,1])))
    

### Generate a vertical latex table of the results (Table 1, 2)

In [ ]:
for KIdx,K in enumerate(K_all):     
    print('\n%--------------------------------------------------------------------')
    print(f'% K: {K}:')    
    print('%--------------------------------------------------------------------')

    print('\\begin{table}[]\n\\footnotesize\\centering\n\\resizebox{0.75\\textwidth}{!}{\n\\begin{tabular}{@{}rccccccc@{}}\n\\toprule')
                          
    # headers
    line = '{:27} & {:20}'.format('', '$r$')
    for methodIdx, method in enumerate(methods):
        
        method = method.replace('_', ' ')                
        if method == 'heavy edge':
            method = '\\begin{tabular}[c]{@{}c@{}}heavy\\\\ edge\\end{tabular}'
        elif 'variation edges' in method:
            method = '\\begin{tabular}[c]{@{}c@{}}local var.\\\\ (edges)\\end{tabular}'
        elif (method == 'variation neighborhoods') or (method == 'variation neighborhood'):
            method = '\\begin{tabular}[c]{@{}c@{}}local var.\\\\ (neigh.)\\end{tabular}'
        elif 'algebraic' in method:
            method = '\\begin{tabular}[c]{@{}c@{}}algebraic\\\\ distance\\end{tabular}'
        elif 'affinity' in method:
            method = 'affinity'
        elif method == 'kron':
            method = '\\begin{tabular}[c]{@{}c@{}}Kron\\\\ reduction\\end{tabular}'
        else: continue
            
        line += ' & {:20}'.format(method)
    line += '\\\\ \\midrule'
    print(line)
    

    for graphIdx, graph in enumerate(graphs):       
        
        filepath = os.path.join('.', 'results', 'experiment_spectrum_'+ graph +'.npz')
        data = np.load(filepath)
        eigenvalue = data['eigenvalue']#*lmax[graphIdx]

        for rIdx, r in enumerate(r_all):
            if rIdx == 0: line = '\\multirow{3}{*}{' +  graph + '}'
            else: line = ''
            line = '{:27} & {:19}\%'.format(line, int(r*100))
            
            for methodIdx, method in enumerate(methods):
                
                if min(eigenvalue[:,KIdx,rIdx]) == eigenvalue[methodIdx,KIdx,rIdx]:
                    line += ' & \\textbf{{{:0.3f}}}{:6}'.format(eigenvalue[methodIdx,KIdx,rIdx],'')
                else:
                    line += ' & {:0.3f}{:15}'.format(eigenvalue[methodIdx,KIdx,rIdx], '')
            
            line += '\\\\'
            if rIdx == len(r_all)-1 and graphIdx < len(graphs)-1: line += '\cmidrule(l){2-8}'
            print(line)
            
    
    print('\\bottomrule\n\\end{tabular}\n}\n\\caption{??}\n\\label{table:K=' +  str(K) + '}\n\\end{table}')